## Assignment 07: Mustaeen Ahmed

In [1]:
import pygame
import moderngl
import numpy
from pyglm import glm
from loadModelUsingAssimp_V1 import getObjectDataList

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
geomDataList, indexDataList, bound, texNames, scene = getObjectDataList("mario_obj/scene.gltf", True)

SCENE:
number of meshes: 8
number of materials: 9
Mesh Name:  Object_0
material Index:  0
number of vertices:  1801
number of faces:  3322
number of indices:  9966
first 3 verts:
[ 0.29 -1.18  6.59]
minimum:  -1.815517
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  3602
first 3 texture coordinates:
[2.94 1.23 2.95]
Mesh Name:  Object_1
material Index:  1
number of vertices:  3496
number of faces:  6616
number of indices:  19848
first 3 verts:
[-0.67 -0.91  5.68]
minimum:  -1.465921
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  6992
first 3 texture coordinates:
[1.75 1.74 1.76]
Mesh Name:  Object_2
material Index:  2
number of vertices:  3738
number of faces:  6380
number of indices:  19140
first 3 verts:
[2.65 0.09 4.35]
minimum:  -3.850452
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  7476
first 3 texture coordinates:
[5.41 1.98 5.41]


#### Intialize everything

In [ ]:
pygame.init()

width = 840
height = 480
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Assignment 07: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info["GL_VERSION"]

'4.6.0 - Build 32.0.101.6881'

#### Create shader program(s) and program

In [4]:
#
# Vertex shader(s)
#
vertex_shader_code = '''
#version 330 core
layout (location=0) in vec3 position;
layout (location=1) in vec2 uv;

uniform mat4 model, view, perspective;
out vec2 f_uv; // Texture coordinate

void main() {
    f_uv = uv;
    vec4 P = perspective*view*model*vec4(position, 1);
    gl_Position = P;
}
'''

#
# Fragment shader(s)
#
fragment_shader_code = '''
#version 330 core
in vec2 f_uv;
uniform sampler2D map;
// Add output variable here
out vec4 color;

void main() {
    //set output color value here
    color = texture(map, f_uv);
}
'''
#
# Programs
#

program = gl.program(
    vertex_shader= vertex_shader_code,
    fragment_shader= fragment_shader_code
)

#### The renderables

In [5]:
# geomDataList, indexDataList
renderables = []
for i, geomData in enumerate(geomDataList):

    geomBuffer = gl.buffer(geomData)
    indexBuffer = gl.buffer(indexDataList[i])

    renderable = gl.vertex_array(program,
        [(geomBuffer, "3f 2f", "position", "uv")], index_buffer=indexBuffer,index_element_size=4
    )
    renderables.append(renderable)

#### Array of the textures

In [6]:
texNames

['mario_obj/textures/submesh_0_baseColor.png',
 'mario_obj/textures/submesh_1_baseColor.png',
 'mario_obj/textures/submesh_2_baseColor.png',
 'mario_obj/textures/submesh_3_baseColor.png',
 'mario_obj/textures/submesh_4_baseColor.png',
 'mario_obj/textures/submesh_5_baseColor.png',
 'mario_obj/textures/submesh_6_baseColor.png',
 'mario_obj/textures/submesh_7_baseColor.png']

#### Create texture objs and specify filter/wrap options

In [7]:
# texNames
for i, texName in enumerate(texNames): 
    texture_img = pygame.image.load(texName) # clock face change to texName
    texture_data = pygame.image.tobytes(texture_img,"RGB", True) 
    texture = gl.texture(texture_img.get_size(), data = texture_data, components=3)
    texture.build_mipmaps()
    sampler = gl.sampler(texture=texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x = True, repeat_y = True)
    sampler.use(i)

#### Define camera parameters

In [8]:
displacement_vector = 2*bound.radius*glm.vec3(0,0,1) #glm.rotate(glm.vec3(0,1,0), glm.radians(60), glm.vec3(1,0,0)) #
    
target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0,1,0)

width = 840
height = 480
### View volume parameters
fov_radian = glm.radians(45) # In radian
aspect = width/height
near = bound.radius
far = 3*bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

#### Render loop

In [ ]:
def recursive_render(node, M):
    if node is None:
        print("Empty node")
        return
    #print("Number of meshes: ",len(node.meshes))
    nodeTransform = glm.transpose(glm.mat4(node.transformation));
    currentTransform = M * nodeTransform
    if node.num_meshes > 0:
        for index in node.mesh_indices:
            program["map"] = index
            program["model"].write(currentTransform)
            renderables[index].render()

    for node in node.children:
        recursive_render(node, currentTransform)

def render():
    recursive_render(scene.root_node, glm.mat4(1))

running = True
clock = pygame.time.Clock()
alpha = 0
pause = True
gl.enable(gl.DEPTH_TEST)
while running:   
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    # event.key == 27 means Escape key is pressed.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type ==  pygame.KEYDOWN):
            if  event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width/height, near, far)

    # create the aspect ratio correction matrix
    
    eye_point = glm.vec3(bound.center) + glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0,1,0))
    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    
    ### Add render code below
    # Clear the display window with the specified R, G, B values using function ctx.clear(R, G, B)
    gl.clear(0.5,0.5,0.0)
    
    # Make one or more Render Calls to instruct the GPU to render by executing the shader program with the provided data.
    program["view"].write(viewMatrix)
    program["perspective"].write(perspectiveMatrix)

    render()

    pygame.display.flip()
    # press P to spin the obj 
    if not pause:
        clock.tick(60)  # limits FPS to 10
        alpha = alpha + 1
        if alpha > 360:
            alpha = 0
    
pygame.display.quit()